In [1]:
import os
from typing import List, Dict, Any
import pandas as pd

In [2]:
from langchain_core.documents import Document
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
)
print("Imports successful")

Imports successful


## Understanding Documents in LangChain

In [3]:
##Understanding Documents in LangChain
doc = Document(
    page_content="This is the content of the document.",
    metadata={"source":"exmaple.txt",
        "pages":1,
        "author":"Prithviraj Acharya",
        "date_created":"2025-01-01",
        "custom_key":"custom_value"
        },
)

print("Document created successfully.")
print(f"Document Source: {doc.page_content}")
print(f"Document metadata: {doc.metadata}")

Document created successfully.
Document Source: This is the content of the document.
Document metadata: {'source': 'exmaple.txt', 'pages': 1, 'author': 'Prithviraj Acharya', 'date_created': '2025-01-01', 'custom_key': 'custom_value'}


In [4]:
## Create a simple txt file
import os
os.makedirs("../data/text_files",exist_ok=True)

In [5]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


## Document Loaders

In [6]:
# Text loader - Read single file

from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/text_files/python_intro.txt")
documents = loader.load()       

print(f"Loaded {len(documents)} document(s).")
print(f"type of document: {type(documents[0])}")
print(f"Document content preview:\n{documents[0].page_content[:200]}...")
print(f"Document metadata:\n{documents[0].metadata}")



Loaded 1 document(s).
type of document: <class 'langchain_core.documents.base.Document'>
Document content preview:
Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has b...
Document metadata:
{'source': '../data/text_files/python_intro.txt'}


In [7]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt", ## Pattern to match files  
    loader_cls= TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=True

)

documents=dir_loader.load()

for i,doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"Content Preview: {doc.page_content[:100]}...")
    print(f"Source: {doc.metadata['source']}")
    print(f"Type: {type(doc)}")
    print(f'Length of document: {len(doc.page_content)} characters')
print(f"\nTotal documents loaded: {len(documents)}")

100%|██████████| 2/2 [00:00<00:00, 2126.93it/s]


Document 1:
Content Preview: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables system...
Source: ..\data\text_files\machine_learning.txt
Type: <class 'langchain_core.documents.base.Document'>
Length of document: 575 characters

Document 2:
Content Preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
Source: ..\data\text_files\python_intro.txt
Type: <class 'langchain_core.documents.base.Document'>
Length of document: 489 characters

Total documents loaded: 2


## Text Splitting

In [8]:
# Method 1 - character-based splitting

text = documents[0].page_content

print('1. CharacterTextSplitter')
char_splitter = CharacterTextSplitter(
    separator="\n", # split on new line character
    chunk_size=200, # max chunk size in characters
    chunk_overlap=20, # max overlap between chunks in characters
    length_function=len # How to measure chunk size
)

char_chunks = char_splitter.split_text(text)

print(f"Total chunks created: {len(char_chunks)}")
print(f"first chunk preview::\n{char_chunks[0][:200]}...")



1. CharacterTextSplitter
Total chunks created: 4
first chunk preview::
Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems to learn and improve...


In [9]:
# Method 2 - Recursive Character-based splitting (Most Recommended)

text = documents[0].page_content

print('2. RecursiveCharacterTextSplitter')
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""], # split on multiple separators in order
    chunk_size=200, # max chunk size in characters
    chunk_overlap=20, # max overlap between chunks in characters
    length_function=len # How to measure chunk size
)

recursive_chunks = recursive_splitter.split_text(text)

print(f"Total chunks created: {len(recursive_chunks)}")
print(f"first chunk preview::\n{recursive_chunks[0][:200]}...")

2. RecursiveCharacterTextSplitter
Total chunks created: 6
first chunk preview::
Machine Learning Basics...


In [10]:
print(recursive_chunks[0])
print("--------------------")
print(recursive_chunks[1])
print("--------------------")
print(recursive_chunks[2])
print("--------------------")
print(recursive_chunks[3])
print("--------------------")
print(recursive_chunks[4])
print("--------------------")

Machine Learning Basics
--------------------
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
--------------------
that can access data and use it to learn for themselves.
--------------------
Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
--------------------
3. Reinforcement Learning: Learning through rewards and penalties
--------------------


In [11]:
#  Method 3 - Token-based splitting

text = documents[0].page_content
print('3. TokenTextSplitter')
token_splitter = TokenTextSplitter(
    chunk_size=50, # max chunk size in tokens
    chunk_overlap=10, # max overlap between chunks in tokens
)

token_chunks = token_splitter.split_text(text)

print(f"Total chunks created: {len(recursive_chunks)}")
print(f"first chunk preview::\n{recursive_chunks[0][:200]}...")

3. TokenTextSplitter
Total chunks created: 6
first chunk preview::
Machine Learning Basics...
Total chunks created: 6
first chunk preview::
Machine Learning Basics...


In [12]:
# Comparison of different splitting methods

print('📊 Text Splitter Comparison:')

print('\n🔄 RecursiveCharacterTextSplitter:')
print('✅ Pros: Smart splitting, preserves structure, best for RAG')
print('❌ Cons: Slower, character-based not token-aware')

print('\n📏 CharacterTextSplitter:')
print('✅ Pros: Fast, predictable sizes, simple')
print('❌ Cons: Breaks text awkwardly, ignores boundaries')

print('\n🎫 TokenTextSplitter:')
print('✅ Pros: Token-aware, respects word boundaries, LLM-optimized')
print('❌ Cons: Slower, tokenizer-dependent, less intuitive')

print('\n💡 Recommendation: Use RecursiveCharacterTextSplitter for most cases')

# Results
print(f"\n📈 Results:")
print(f"Character: {len(char_chunks)} chunks, {sum(len(c) for c in char_chunks)} chars")
print(f"Recursive: {len(recursive_chunks)} chunks, {sum(len(c) for c in recursive_chunks)} chars")
print(f"Token: {len(token_chunks)} chunks, {sum(len(c) for c in token_chunks)} chars")

# Safe preview function
def safe_preview(text, length=100):
    try:
        return text[:length] + "..." if len(text) > length else text
    except UnicodeEncodeError:
        # Remove problematic characters
        return ''.join(c for c in text[:length] if ord(c) < 0x10000) + "..."

print(f"\n👀 Previews:")
print(f"Character: {safe_preview(char_chunks[0])}")
print(f"Recursive: {safe_preview(recursive_chunks[0])}")
print(f"Token: {safe_preview(token_chunks[0])}")

📊 Text Splitter Comparison:

🔄 RecursiveCharacterTextSplitter:
✅ Pros: Smart splitting, preserves structure, best for RAG
❌ Cons: Slower, character-based not token-aware

📏 CharacterTextSplitter:
✅ Pros: Fast, predictable sizes, simple
❌ Cons: Breaks text awkwardly, ignores boundaries

🎫 TokenTextSplitter:
✅ Pros: Token-aware, respects word boundaries, LLM-optimized
❌ Cons: Slower, tokenizer-dependent, less intuitive

💡 Recommendation: Use RecursiveCharacterTextSplitter for most cases

📈 Results:
Character: 4 chunks, 562 chars
Recursive: 6 chunks, 560 chars
Token: 3 chunks, 666 chars

👀 Previews:
Character: Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems...
Recursive: Machine Learning Basics
Token: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables system...
